# Singapore Road Data
This is a notebook that will guide you through 
1. Obtaining data from openstreetmaps
2. Generating useful output from the data
  - A visualisation of all the points
  - A text file with the coordinates, road names and number of lanes
  - A dumped dictionary and kd-tree, which can be used to quickly get information regarding a single point


In [1]:
# External imports
import overpy as op
import sys
import concurrent.futures

# Internal imports (classes, functions)
from road_point import RoadPoint
from output_text import export_road_points
from output_scatter import make_gmap
from output_kd_tree import make_kdtree
from speed_limit_scraper import get_speed_limit

In [2]:
# Get API
api = op.Overpass()

# Query
query = "".join(l for l in open('query.xml'))
result = api.query(query)

Get all the ways (roads), do some duplicate removal and convert them to `RoadPoint`s. `RoadPoint` is a class used to encompass all the information we want about a coordinate.

In [5]:
# Get ways (roads or part of roads) 
# which have road names and lane count.
# Note that if we exclude the filters, there are many more 
# ponits to process (or even just lane count).
all_ways = result.get_ways()
ways = [w for w in all_ways if 'name' in w.tags and 'lanes' in w.tags]

# Remove duplicates in nodes (e.g 2 connected ways sharing a node)
node_dict = {}
for way in ways:
    for node in way.nodes:
        node_dict[node.id] = node

# Converts a node to a RoadPoint, without speed_limit
# This is done to prevent overloading in the multiprocessing
# step with the large overpy node object.
def node_to_RoadPoint(node):
    lat = float(node.lat)
    lon = float(node.lon)
    road_name = way.tags.get('name')
    lane_count =  way.tags.get('lanes')
    return RoadPoint(node.id, lat, lon, road_name, lane_count) 


unique_nodes = list(node_dict.values())
road_points_without_speedlimit = [node_to_RoadPoint(n) for n in unique_nodes]

Now, we use concurrency to add the speed limit into all the coordinates. We do this because fetching each speed limit takes a while, and it will be hard to process this for all the points sequentially.

In [ ]:
# Process a road point and add a speed limit property.
# Abstracted to enable concurrency for processing nodes.
def append_speed_limit(road_point):
    road_point.speed_limit = get_speed_limit(road_point.road_name)
    print('Fetched speed limit for node ' + str(road_point.node_id))
    return road_point

executor = concurrent.futures.ThreadPoolExecutor()
road_points = list(executor.map(append_speed_limit, road_points_without_speedlimit))
print("done processing all points!")




Speed limit at Hoy Fatt Road: 50
Fetched speed limit for node 4748500346
Speed limit at Hoy Fatt Road: 50
Fetched speed limit for node 3689717011
Speed limit at Hoy Fatt Road: 50
Speed limit at Hoy Fatt Road: 50Fetched speed limit for node 5278609747

Fetched speed limit for node 473019059
Speed limit at Hoy Fatt Road: 50
Speed limit at Hoy Fatt Road: 50
Fetched speed limit for node 4748500344
Speed limit at Hoy Fatt Road: 50
Fetched speed limit for node 1363263251
Fetched speed limit for node 247749632
Speed limit at Hoy Fatt Road: 50
Fetched speed limit for node 74368119
Speed limit at Hoy Fatt Road: 50Speed limit at Hoy Fatt Road: 50
Fetched speed limit for node 74368117

Fetched speed limit for node 5134004761
Speed limit at Hoy Fatt Road: 50
Speed limit at Hoy Fatt Road: 50
Fetched speed limit for node 4737110645
Speed limit at Hoy Fatt Road: 50
Fetched speed limit for node 74371535
Fetched speed limit for node 243235412
Speed limit at Hoy Fatt Road: 50
Fetched speed limit for nod


Fetched speed limit for node 175296375Fetched speed limit for node 241837405Speed limit at Hoy Fatt Road: 50
Fetched speed limit for node 241824679


Speed limit at Hoy Fatt Road: 50
Fetched speed limit for node 241837396
Speed limit at Hoy Fatt Road: 50
Fetched speed limit for node 1711845455
Speed limit at Hoy Fatt Road: 50
Fetched speed limit for node 175296342
Speed limit at Hoy Fatt Road: 50
Fetched speed limit for node 4747767712
Speed limit at Hoy Fatt Road: 50
Speed limit at Hoy Fatt Road: 50Speed limit at Hoy Fatt Road: 50Fetched speed limit for node 1711845459

Fetched speed limit for node 241893460

Speed limit at Hoy Fatt Road: 50Speed limit at Hoy Fatt Road: 50
Fetched speed limit for node 1711845451
Speed limit at Hoy Fatt Road: 50
Fetched speed limit for node 1741870417

Fetched speed limit for node 236507878
Fetched speed limit for node 236516191
Speed limit at Hoy Fatt Road: 50
Fetched speed limit for node 5189333689
Speed limit at Hoy Fatt Road: 50
Speed limit at Hoy

Speed limit at Hoy Fatt Road: 50
Fetched speed limit for node 2429759706
Speed limit at Hoy Fatt Road: 50
Fetched speed limit for node 1669539690
Speed limit at Hoy Fatt Road: 50
Fetched speed limit for node 246491761
Speed limit at Hoy Fatt Road: 50
Fetched speed limit for node 245161700
Speed limit at Hoy Fatt Road: 50
Fetched speed limit for node 1729358975
Speed limit at Hoy Fatt Road: 50
Speed limit at Hoy Fatt Road: 50
Fetched speed limit for node 389448452
Fetched speed limit for node 5134030121
Speed limit at Hoy Fatt Road: 50
Fetched speed limit for node 5129432640
Speed limit at Hoy Fatt Road: 50
Fetched speed limit for node 1670227420
Speed limit at Hoy Fatt Road: 50
Speed limit at Hoy Fatt Road: 50
Fetched speed limit for node 247749739
Speed limit at Hoy Fatt Road: 50
Fetched speed limit for node 2647331479
Speed limit at Hoy Fatt Road: 50
Fetched speed limit for node 247749720
Speed limit at Hoy Fatt Road: 50
Fetched speed limit for node 247749740
Speed limit at Hoy Fatt 